### Decision Tree (의사결정 나무)
- 트리 구조를 사용하고 각 노드에는 분석 대상의 속성(변수)들이 위치
- 각 노드마다 목표 값을 가장 잘 분류할 수 있는 속성을 찾아서 배치
- 해당 속성이 갖는 값을 이용하여 새로운 가지(branch)를 만든다.
- 각 노드에서 최적을 속성을 선택할 때는 해당 속성을 기준으로 분류한 값들이 구분되는 정도 측정
- 다른 종류의 값들이 섞여 있는 정도롤 나타내는 Entropy를 주로 활용
- Entropy가 낮을수록 분류가 잘 된 것 
- Entropy가 일정 수준 이하로 낮아질 때까지는 앞 과정 반복

In [1]:
# 데이터 준비
# 암세포 진단(breast cancer) 데이터셋 사용

import pandas as pd
import numpy as np

uci_path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/\
breast-cancer-wisconsin/breast-cancer-wisconsin.data'

df = pd.read_csv(uci_path, header=None)

In [2]:
df.columns = ['id', 'clump', 'cell_size', 'cell_shape', 'adhesion', 'epithlial',
            'bare_nuclei', 'chromatin', 'normal_nucleoli', 'mitoses', 'class']

pd.set_option('display.max_columns', 15)

In [4]:
# Step-2 : 데이터 탐색

print(df.head())
print('\n')

        id  clump  cell_size  cell_shape  adhesion  epithlial bare_nuclei  \
0  1000025      5          1           1         1          2           1   
1  1002945      5          4           4         5          7          10   
2  1015425      3          1           1         1          2           2   
3  1016277      6          8           8         1          3           4   
4  1017023      4          1           1         3          2           1   

   chromatin  normal_nucleoli  mitoses  class  
0          3                1        1      2  
1          3                2        1      2  
2          3                1        1      2  
3          3                7        1      2  
4          3                1        1      2  




In [5]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               699 non-null    int64 
 1   clump            699 non-null    int64 
 2   cell_size        699 non-null    int64 
 3   cell_shape       699 non-null    int64 
 4   adhesion         699 non-null    int64 
 5   epithlial        699 non-null    int64 
 6   bare_nuclei      699 non-null    object
 7   chromatin        699 non-null    int64 
 8   normal_nucleoli  699 non-null    int64 
 9   mitoses          699 non-null    int64 
 10  class            699 non-null    int64 
dtypes: int64(10), object(1)
memory usage: 60.2+ KB
None


In [6]:
print(df.describe())

                 id       clump   cell_size  cell_shape    adhesion  \
count  6.990000e+02  699.000000  699.000000  699.000000  699.000000   
mean   1.071704e+06    4.417740    3.134478    3.207439    2.806867   
std    6.170957e+05    2.815741    3.051459    2.971913    2.855379   
min    6.163400e+04    1.000000    1.000000    1.000000    1.000000   
25%    8.706885e+05    2.000000    1.000000    1.000000    1.000000   
50%    1.171710e+06    4.000000    1.000000    1.000000    1.000000   
75%    1.238298e+06    6.000000    5.000000    5.000000    4.000000   
max    1.345435e+07   10.000000   10.000000   10.000000   10.000000   

        epithlial   chromatin  normal_nucleoli     mitoses       class  
count  699.000000  699.000000       699.000000  699.000000  699.000000  
mean     3.216023    3.437768         2.866953    1.589413    2.689557  
std      2.214300    2.438364         3.053634    1.715078    0.951273  
min      1.000000    1.000000         1.000000    1.000000    2.0000

In [7]:
# object형인 bare_nuclei 열을 숫자형으로 변환하고 
# unique() 메소드로 고유값 확인 후 '?'을 np.nan으로 바꿔주고 삭제한다
df['bare_nuclei'].unique()

array(['1', '10', '2', '4', '3', '9', '7', '?', '5', '8', '6'],
      dtype=object)

In [8]:
df['bare_nuclei'].replace('?', np.nan, inplace= True)
df.dropna(subset=['bare_nuclei'], axis = 0, inplace=True)
df['bare_nuclei'] = df['bare_nuclei'].astype('int')

In [9]:
df.describe()

,id,clump,cell_size,cell_shape,adhesion,epithlial,bare_nuclei,chromatin,normal_nucleoli,mitoses,class
count,6.830000e+02,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000
mean,1.076720e+06,4.442167,3.150805,3.215227,2.830161,3.234261,3.544656,3.445095,2.869693,1.603221,2.699854
std,6.206440e+05,2.820761,3.065145,2.988581,2.864562,2.223085,3.643857,2.449697,3.052666,1.732674,0.954592
min,6.337500e+04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000
25%,8.776170e+05,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000
50%,1.171795e+06,4.000000,1.000000,1.000000,1.000000,2.000000,1.000000,3.000000,1.000000,1.000000,2.000000
75%,1.238705e+06,6.000000,5.000000,5.000000,4.000000,4.000000,6.000000,5.000000,4.000000,1.000000,4.000000
max,1.345435e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


- 데이터프레임에서 설명 변수 X로 사용할 열들을 먼저 선택 
- 예측 변수로 사용할 'class'열 선택 
- 설명 변수 정규화 후-> 훈련 데이터와 검증 데이터 분리

In [10]:
# Step-3 : 데이터셋 구분 - train / test 

X = df[['clump', 'cell_size', 'cell_shape', 'adhesion', 'epithlial',
            'bare_nuclei', 'chromatin', 'normal_nucleoli', 'mitoses']]
y = df['class']

from sklearn import preprocessing 
X = preprocessing.StandardScaler().fit(X).transform(X)

from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

print('train : ', X_train.shape)
print('test : ', X_test.shape)

train :  (478, 9)
test :  (205, 9)


### 모형 학습 및 검증 
- sklearn 라이브러리의 tree모듈 import 한다. 
- DecisionTreeClassifier() 함수를 사용하여 객체 생성 
- 분류 정도를  평가하는 기준으로 'entropy' 사용 
- 트리 레벨을 5로 지정 -> 5단계까지 가지를 확장 할 수 있다는 뜻
- 레벨이 많아질수록 훈련 데이터에 대한 예측은 정확해짐 
- 하지만 훈련 데이터에 지나치게 최적화 되어 오버피팅 문제 발생 

In [12]:
# Step-4 : Decision Tree 분류 모형 
from sklearn import tree  

tree_model = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth=1)

tree_model.fit(X_train, y_train)

y_hat = tree_model.predict(X_test) 

print(y_hat[0:10])
print(y_test.values[0:10])

[4 2 2 4 4 4 2 2 4 4]
[4 4 4 4 4 4 2 2 4 4]


- 모형 평가 지표 계산
- 양성 종양 목표 값 2, 악성 종양 목표 값 4 
- classification_report() 함수로 precision, recall, f1 - score 지표 출력 

In [13]:
from sklearn import metrics
tree_matrix = metrics.confusion_matrix(y_test, y_hat)
print(tree_matrix)
print('\n')

tree_report = metrics.classification_report(y_test, y_hat)
print(tree_report)

[[128   3]
 [  9  65]]


              precision    recall  f1-score   support

           2       0.93      0.98      0.96       131
           4       0.96      0.88      0.92        74

    accuracy                           0.94       205
   macro avg       0.95      0.93      0.94       205
weighted avg       0.94      0.94      0.94       205

